# Step 1: Generating Negotaition Scenarios code
## Project setup
This notebook generates negotiation scenarios and their corresponding transition matrices. You can either upload your own CSV following the required format or create scenarios randomly using preset parameters. To begin, run the first cell below.

### Part 1: Run the Setup Cell below
This step installs the needed packages, loads all imports, and initializes the project environment. It detects whether you are in Google Colab or running locally, prepares the data/ and outputs/ folders, and mounts Google Drive when needed so all inputs and outputs stay organized in the same structure.

In [ ]:
# Install packages if you don't have them yet
!pip install numpy
!pip install pandas
!pip install tqdm

# Import packages
import os
from pathlib import Path
from __future__ import annotations
import os
import random
import warnings
from itertools import product
from typing import List, Tuple, Optional
import numpy as np
import pandas as pd
from tqdm import tqdm

# ── silence pandas chatter ────────────────────────────────────────────
from pandas.errors import SettingWithCopyWarning
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=SettingWithCopyWarning)

def in_colab():
    return "COLAB_RELEASE_TAG" in os.environ or "COLAB_GPU" in os.environ

# Default: where the notebook is saved
PROJECT_ROOT = Path.cwd()

if in_colab():
    from google.colab import drive  # type: ignore
    drive.mount("/content/drive", force_remount=False)

    NEW_PATH = Path("/content/drive/MyDrive/Negotiations-as-Search")
    if NEW_PATH.exists():
        PROJECT_ROOT = NEW_PATH

print("Project root:", PROJECT_ROOT)

# Step 1 folders
S1_RAW     = PROJECT_ROOT / "S1_RAW"
S1_OUTPUT  = PROJECT_ROOT / "S1_OUTPUT"

# Step 2 folders
S2_OUTPUT  = PROJECT_ROOT / "S2_OUTPUT"

# Step 3 folders
S3_OUTPUT  = PROJECT_ROOT / "S3_OUTPUT"

# Make sure they exist
for p in [S1_RAW, S1_OUTPUT, S2_OUTPUT, S3_OUTPUT]:
    p.mkdir(parents=True, exist_ok=True)

print("S1 raw:", S1_RAW)
print("S1 output:", S1_OUTPUT)
print("S2 output:", S2_OUTPUT)
print("S3 output:", S3_OUTPUT)

# Helpers (shortcuts for saving/reading)
def s1raw(path): return S1_RAW / path
def s1out(path): return S1_OUTPUT / path
def s2out(path): return S2_OUTPUT / path
def s3out(path): return S3_OUTPUT / path


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Project root: /content/drive/MyDrive/Negotiations-as-Search
S1 raw: /content/drive/MyDrive/Negotiations-as-Search/S1_RAW
S1 output: /content/drive/MyDrive/Negotiations-as-Search/S1_OUTPUT
S2 output: /content/drive/MyDrive/Negotiations-as-Search/S2_OUTPUT
S3 output: /content/drive/MyDrive/Negotiations-as-Search/S3_OUTPUT


# Code for Generating Negotiation Scenarios

### Part 2: Helper functions
Run this code to load the support/helper functions used for scenario generation.

In [ ]:
"""
transition_analysis.py
================================================
Generate **transition‑matrix CSVs** either by:

* Random mode (default) – build synthetic negotiation scenarios for
  every (N, M) combination; optional ZOPA filter keeps only scenarios
  with a consensus node (uses caller‑supplied set_agreement_threshold).

* Manual mode – convert existing payoff CSVs (long on/off format).

Console output
--------------
• Outer tqdm bar – files (manual) or scenarios (random).
• Inner single‑line bar – payoff enumeration for each file.
• After each CSV is written a one‑line confirmation is printed:
  ✔ Saved <file> → <path>

Columns in every transition CSV
-------------------------------
Worse_Off  – parties whose payoff drops in the transition.
Accepting  – parties whose payoff in *destination* ≥ reservation.

Party names in **manual mode** are shortened to acronyms
(initial caps of each word).
All pandas warnings are silenced.
"""



# ── helper utilities ──────────────────────────────────────────────────
def _acronyms(names: List[str]) -> List[str]:
    """e.g. 'Stellar Cove' → 'SC'."""
    return ["".join(w[0] for w in name.split() if w[0].isupper()) or name[:3]
            for name in names]


def _expand_payoff_df(
    payoff: np.ndarray, N: int, M: int, *, RP: int = 0, TP: int = 2
) -> pd.DataFrame:
    cols = [f"Party_{i+1}" for i in range(N)]
    df = pd.DataFrame(payoff,
                      index=[f"Issue_{j+1}" for j in range(M)],
                      columns=cols)
    df.loc["reservation"] = [RP] * N
    df.loc["target"]      = [TP] * N

    df.reset_index(inplace=True)
    df.rename(columns={"index": "issue"}, inplace=True)

    on  = df.copy(); on["option"] = "on"
    off = df.copy(); off["option"] = "off"; off.loc[:, cols] = 0
    on.loc[on["issue"].isin(["reservation", "target"]), "option"] = "N/A"
    off = off[~off["issue"].isin(["reservation", "target"])]

    full = pd.concat([on, off]).sort_values(
        ["issue", "option"], ascending=[True, False]
    )
    return full[["issue", "option", *cols]].reset_index(drop=True)


def _to_dfn(df_expanded: pd.DataFrame) -> Tuple[pd.DataFrame, List[str]]:
    """Return DF‑N (analysis‑ready) and list of party names."""
    dfn     = df_expanded.copy()
    parties = dfn.columns[2:].tolist()

    res = dfn[dfn["issue"] == "reservation"]
    tgt = dfn[dfn["issue"] == "target"]
    core = dfn[~dfn["issue"].isin(["reservation", "target"])]

    core["option-issue code"] = core.groupby("issue").cumcount() + 1
    issue_map = {iss: idx for idx, iss in enumerate(core["issue"].unique(), 1)}
    core["issue code"] = core["issue"].map(issue_map).astype("Int64")

    cols = ["issue", "issue code", "option", "option-issue code", *parties]
    dfn_final = pd.concat([core[cols], res, tgt], ignore_index=True)
    return dfn_final, parties

def set_agreement_threshold(nego_df, N, rule_choice):
    """
    Add Origin_Agreement and Destination_Agreement flags.

    rule_choice
        1  – simple majority (> N/2)
        2  – max‑votes‑minus‑one
        3  – unanimity (default used in your calls)
    """
    if rule_choice == 1:
        threshold = N // 2
        nego_df["Origin_Agreement"]      = (nego_df["Origin_votes"]      > threshold).astype(int)
        nego_df["Destination_Agreement"] = (nego_df["Destination_votes"] > threshold).astype(int)

    elif rule_choice == 2:
        thresh = nego_df["Origin_votes"].max() - 1
        nego_df["Origin_Agreement"]      = (nego_df["Origin_votes"]      >= thresh).astype(int)
        nego_df["Destination_Agreement"] = (nego_df["Destination_votes"] >= thresh).astype(int)

    elif rule_choice == 3:
        nego_df["Origin_Agreement"]      = (nego_df["Origin_votes"]      == N).astype(int)
        nego_df["Destination_Agreement"] = (nego_df["Destination_votes"] == N).astype(int)

    else:
        raise ValueError("rule_choice must be 1, 2, or 3.")

    return nego_df          # ← ***add this line***


# ── full transition analysis for a single scenario ───────────────────
def _full_transition_analysis(
    dfn: pd.DataFrame,
    parties: List[str],
    *,
    rng: random.Random,
    sampling_proportion: float = 1.0,
    inner_bar: Optional[tqdm] = None,
) -> pd.DataFrame:
    """Build the full origin→destination transition matrix for one scenario.

    * Computes every proposal’s payoff (or a subsample).
    * Adds `Worse_Off` and `Accepting` columns.
    * Works even when party names contain spaces.
    * inner_bar (manual mode) updates in a **single line**.
    """
    reservation, target = dfn.iloc[-2], dfn.iloc[-1]
    core = dfn.iloc[:-2].sort_values(["issue code", "option-issue code"])

    # enumerate proposals -------------------------------------------------
    issues = core["issue code"].unique()
    opts = {
        i: core.loc[core["issue code"] == i, "option-issue code"]
              .astype(int).tolist()
        for i in issues
    }
    combos = list(product(*opts.values()))
    rng.shuffle(combos)
    combos = combos[: max(1, int(len(combos) * sampling_proportion))]

    dfp = pd.DataFrame(combos, columns=issues)
    dfp["proposal"] = ["".join(map(str, map(int, c))) for c in combos]

    # compute payoffs with optional inner progress bar -------------------
    if inner_bar is not None:
        inner_bar.reset(total=len(dfp))

    for idx in range(len(dfp)):
        prop = dfp.at[idx, "proposal"]
        rows = core.set_index(
            ["issue code", "option-issue code"]
        ).loc[list(zip(issues, map(int, prop)))]
        for pt in parties:
            dfp.at[idx, f"{pt}_payoff"] = rows[pt].sum()

        if inner_bar is not None:
            inner_bar.update(1)

    # prepare for transition matrix ---------------------------------------
    # compact dict: proposal → {payoffs, votes}
    for pt in parties:
        dfp[f"{pt}_meets_reservation"] = dfp[f"{pt}_payoff"] >= reservation[pt]
    dfp["Votes"] = dfp[[f"{pt}_meets_reservation" for pt in parties]].sum(axis=1)
    prop_dict = {
        row["proposal"]: {
            "p": {pt: row[f"{pt}_payoff"] for pt in parties},
            "v": row["Votes"],
        }
        for _, row in dfp.iterrows()
    }

    # build transition rows with progress bar -----------------------------
    total_transitions = len(prop_dict) * (len(prop_dict) - 1)
    pos = inner_bar.pos + 1 if inner_bar is not None else 0
    trans_bar = tqdm(total=total_transitions,
                     desc="Transitions", ncols=100,
                     position=pos, leave=False)
    rows = []
    for o, d in product(prop_dict, prop_dict):
        if o == d:
            continue
        od, dd = prop_dict[o], prop_dict[d]
        ovec, dvec = np.fromiter(o, int), np.fromiter(d, int)

        row = {
            "Origin": o,
            "Destination": d,
            "Origin_payoff":      sum(od["p"].values()),
            "Destination_payoff": sum(dd["p"].values()),
            "Origin_votes":       od["v"],
            "Destination_votes":  dd["v"],
            "Transition": f"{o}_{d}",
            "Option_Change": (ovec != dvec).sum(),
            "Distance":      np.linalg.norm(ovec - dvec),
        }
        row["Payoff_Change"] = row["Destination_payoff"] - row["Origin_payoff"]
        row["Vote_Change"]   = row["Destination_votes"]  - row["Origin_votes"]

        worse  = [pt for pt in parties if dd["p"][pt] <  od["p"][pt]]
        accept = [pt for pt in parties if dd["p"][pt] >= reservation[pt]]
        row["Worse_Off"] = ", ".join(worse)  or "None"
        row["Accepting"] = ", ".join(accept) or "None"

        for pt in parties:
            row[f"{pt}_origin_payoff"]      = od["p"][pt]
            row[f"{pt}_destination_payoff"] = dd["p"][pt]

        rows.append(row)
        trans_bar.update(1)

    trans_bar.close()

    tdf = pd.DataFrame(rows)
    tdf["Max Distance"] = tdf.apply(
        lambda r: max(abs(int(a) - int(b))
                      for a, b in zip(r["Origin"], r["Destination"])),
        axis=1,
    )
    tdf["parties_count"] = len(parties)
    tdf["issue_count"]   = len(issues)

    lead = ["Transition", "Payoff_Change", "Vote_Change",
            "Option_Change", "Distance", "Origin", "Destination"]
    rest = [c for c in tdf.columns if c not in lead]
    return tdf[lead + rest]


# ── CSV loader (manual mode) ──────────────────────────────────────────
def _process_uploaded_csv(
    path: str, *, acronymise: bool = True
) -> Tuple[pd.DataFrame, List[str]]:
    dfn = pd.read_csv(path)
    dfn.columns = dfn.columns.str.strip()
    issue_col, option_col = dfn.columns[:2]
    party_cols = dfn.columns[2:].tolist()

    if acronymise:
        ac = _acronyms(party_cols)
        dfn.rename(columns=dict(zip(party_cols, ac)), inplace=True)
        party_cols = ac

    res  = dfn[dfn[issue_col] == "reservation"]
    tgt  = dfn[dfn[issue_col] == "target"]
    core = dfn[~dfn[issue_col].isin(["reservation", "target"])]

    core["option-issue code"] = core.groupby(issue_col).cumcount() + 1
    issue_map = {iss: idx for idx, iss in enumerate(core[issue_col].unique(), 1)}
    core["issue code"] = core[issue_col].map(issue_map).astype("Int64")

    cols = [issue_col, "issue code", option_col,
            "option-issue code", *party_cols]
    return pd.concat([core[cols], res, tgt], ignore_index=True), party_cols

# ── public API ────────────────────────────────────────────────────────
def batch_transition_analysis(
    *,
    # manual‑mode
    manual_upload: bool = False,
    input_dir: str | None = None,
    file_names: List[str] | None = None,
    output_dir: str | None = None,
    acronymise_parties: bool = True,
    # random‑mode
    Ns: List[int] | None = None,
    Ms: List[int] | None = None,
    target_count: int | None = None,
    raw_dir: str | None = None,
    out_dir: str | None = None,
    rule_choice: int = 3,
    require_zopa: bool = True,
    # common
    sampling_proportion: float = 1.0,
    seed: int = 42,
) -> pd.DataFrame:
    """
    Generate or process negotiation scenarios and export transition matrices.

    Random mode (default)
    ---------------------
    ```python
    batch_transition_analysis(
        Ns=[...], Ms=[...], target_count=..., raw_dir="...", out_dir="...",
        require_zopa=True, rule_choice=3
    )
    ```

    Manual mode
    -----------
    ```python
    batch_transition_analysis(
        manual_upload=True,
        input_dir="...", file_names=[...], output_dir="..."
    )
    ```
    """
    rng = random.Random(seed)
    np.random.seed(seed)

    # ─────────── manual mode ───────────
    if manual_upload:
        if not (input_dir and output_dir and file_names):
            raise ValueError("input_dir, output_dir, file_names are required in manual mode.")

        os.makedirs(output_dir, exist_ok=True)
        outer = tqdm(file_names, desc="Files", ncols=100, position=0)
        last_df: Optional[pd.DataFrame] = None

        for fname in outer:
            dfn, parties = _process_uploaded_csv(
                os.path.join(input_dir, fname),
                acronymise=acronymise_parties,
            )
            inner = tqdm(total=0, desc="payoffs", ncols=100,
                         position=1, leave=False)
            last_df = _full_transition_analysis(
                dfn, parties, rng=rng,
                sampling_proportion=sampling_proportion,
                inner_bar=inner,
            )
            inner.close()

            out_path = os.path.join(
                output_dir, f"{os.path.splitext(fname)[0]}_transition_analysis.csv"
            )
            last_df.to_csv(out_path, index=False)
            print(f"✔ Saved {fname} → {out_path}")

        outer.close()
        return last_df  # never None here

    # ─────────── random mode ───────────
    # ─────────── random mode ───────────
    if not all([Ns, Ms, target_count, raw_dir, out_dir]):
        raise ValueError(
            "Ns, Ms, target_count, raw_dir and out_dir are required in random mode."
        )

    os.makedirs(raw_dir, exist_ok=True)
    os.makedirs(out_dir, exist_ok=True)

    total = len(Ns) * len(Ms) * target_count
    outer = tqdm(total=total, desc="Scenarios", ncols=100)
    last_df: Optional[pd.DataFrame] = None

    for N in Ns:
        for M in Ms:
            # -------- ZOPA‑filtered path ---------------------------------
            # -------- ZOPA-filtered path ---------------------------------
            if require_zopa:
                if "set_agreement_threshold" not in globals():
                    raise NameError(
                        "require_zopa=True but set_agreement_threshold() is undefined."
                    )
                sat = globals()["set_agreement_threshold"]  # alias once

                success = redo = attempts = 0
                while success < target_count:
                    attempts += 1

                    payoff = np.random.uniform(-1, 1, (M, N)).round(2)
                    raw = _expand_payoff_df(payoff, N, M)
                    raw.to_csv(
                        os.path.join(raw_dir, f"raw_N={N}_M={M}_{attempts}.csv"),
                        index=False,
                    )

                    dfn, parties = _to_dfn(raw)
                    trans_df = _full_transition_analysis(
                        dfn, parties, rng=rng,
                        sampling_proportion=sampling_proportion,
                    )

                    # ---- check for positive ZOPA -------------------------
                    tmp = sat(trans_df.copy(), len(parties), rule_choice)
                    tmp["Reached Consensus"] = tmp["Destination_Agreement"] == 1

                    # Previously this block filtered out encodings composed entirely of {"2","0"}.
                    # That filter has been removed so that such proposals count as valid consensus nodes.
                    nodes = tmp.loc[tmp["Reached Consensus"], "Destination"].unique().tolist()
                    # ------------------------------------------------------

                    if len(nodes) > 0:
                        out_name = f"UM_N={N}_M={M}_{attempts}.csv"
                        trans_df.to_csv(os.path.join(out_dir, out_name), index=False)
                        success += 1
                        outer.update(1)
                        last_df = trans_df
                    else:
                        redo += 1

            # -------- no ZOPA filtering ----------------------------------
            else:
                for attempt in range(1, target_count + 1):
                    payoff = np.random.uniform(-1, 1, (M, N)).round(2)
                    raw = _expand_payoff_df(payoff, N, M)
                    raw.to_csv(
                        os.path.join(raw_dir, f"raw_N={N}_M={M}_{attempt}.csv"),
                        index=False,
                    )

                    dfn, parties = _to_dfn(raw)
                    trans_df = _full_transition_analysis(
                        dfn, parties, rng=rng,
                        sampling_proportion=sampling_proportion,
                    )
                    trans_df.to_csv(
                        os.path.join(out_dir, f"UM_N={N}_M={M}_{attempt}.csv"),
                        index=False,
                    )
                    last_df = trans_df
                    outer.update(1)

    outer.close()
    return last_df  # last scenario processed



### Part 3: Run code
Use `batch_transition_analysis` with `manual_upload=True` for manual mode (upload an existing payoff matrix CSV; see examples like `FL_new.csv` or `party3_easy.csv` in the `S1_RAW` directory). Set `manual_upload=False` for random mode, where you only specify the parameters below.
- `Ns` for the group size of the scenarios you want to create  
- `Ms` for the number of issues  
- `target_count` for how many scenarios to generate  
- `require_zopa=True` if you only want scenarios with a positive bargaining zone  
- `rule_choice = 3` if agreement requires unanimity  
- `seed` to set the random seed


In [ ]:
# RANDOM MODE
preview = batch_transition_analysis(
    Ns=[2, 6],
    Ms=[8],
    target_count=100,
    raw_dir=S1_RAW,
    out_dir=S1_OUTPUT,
    require_zopa=True,
    rule_choice=3,
    sampling_proportion=1,
    seed=1
)

Scenarios: 100%|██████████████████████████████████████████████████| 200/200 [13:19<00:00,  4.00s/it]


In [ ]:
# MANUAL MODE
preview = batch_transition_analysis(
    manual_upload=True,
    input_dir=S1_RAW,   # drop your manual CSVs here
    file_names=["party3_easy.csv"],
    output_dir=S1_OUTPUT,
    sampling_proportion=1
)

# preview = batch_transition_analysis(
#     manual_upload=True,
#     input_dir=S1_RAW,   # drop your manual CSVs here
#     file_names=["FL_new.csv"],
#     output_dir=S1_OUTPUT,
#     sampling_proportion=1
# )

# # MANUAL MODE
# preview = batch_transition_analysis(
#     manual_upload=True,
#     input_dir=S1_RAW,   # drop your manual CSVs here
#     file_names=["FL_old.csv"],
#     output_dir=S1_OUTPUT,
#     sampling_proportion=1
# )

# # MANUAL MODE
# preview = batch_transition_analysis(
#     manual_upload=True,
#     input_dir=S1_RAW,   # drop your manual CSVs here
#     file_names=["new_recruit_mod.csv"],
#     output_dir=S1_OUTPUT,
#     sampling_proportion=1
# )

Files:   0%|                                                                  | 0/1 [00:00<?, ?it/s]
payoffs: 0it [00:00, ?it/s]
payoffs:  79%|████████████████████████████████████████▏          | 202/256 [00:00<00:00, 356.57it/s]

Files: 100%|██████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.80s/it]

✔ Saved party3_easy.csv → /content/drive/MyDrive/Negotiations-as-Search/S1_OUTPUT/party3_easy_transition_analysis.csv


### OPTIONAL: Code for Generating Additional Scenarios
The code below allows me to generate more scenarios after an initial batch has already been created. For example, if I currently have 2,000 scenarios and want to reach a total of 5,000, this code will generate the additional 3,000 without overwriting the existing ones. Using the original script to generate all 5,000 at once would replace the current dataset, so this version preserves what’s already been generated.

In [ ]:
# --- Copy or Build 5k Scenarios Bundle for Negotiations-as-Search ---

import os, re, shutil, sys
from pathlib import Path
from collections import Counter, defaultdict

# Mount Google Drive (Colab)
try:
    from google.colab import drive  # type: ignore
    drive.mount("/content/drive", force_remount=False)
except Exception:
    pass  # not in Colab

# ------------------------------ CONFIG ------------------------------
MODE = "build_5k_scenarios"   # "copy_from_old_main" or "build_5k_scenarios"

NS = [2]                     # example: [2, 6]
MIN_BASE = 2000                 # seed threshold before generation
TARGET_PER_N = 5000             # final target per N in the new folder

# Paths
DRIVE = "/content/drive/MyDrive"
SRC_MAIN = Path(f"{DRIVE}/ACADS - main - PhD in Management/@Negotiations as Problem Solving/Negotiations_Analysis/CSV_for_analysis/FOA_analysis/Main_Analysis_4")
SRC_RAW  = SRC_MAIN / "raw"

NAS_ROOT = Path(f"{DRIVE}/Negotiations-as-Search")
S1_MAIN  = NAS_ROOT / "S1_OUTPUT"
S1_RAW   = S1_MAIN / "random_raw"

FIVEK_MAIN = NAS_ROOT / "5k_scenarios"
FIVEK_RAW  = FIVEK_MAIN / "random_raw"

for p in [NAS_ROOT, S1_MAIN, S1_RAW, FIVEK_MAIN, FIVEK_RAW]:
    p.mkdir(parents=True, exist_ok=True)

# Progress bar if available
try:
    from tqdm import tqdm
    def _iter_progress(iterable, desc, unit="file"):
        return tqdm(iterable, desc=desc, unit=unit)
except Exception:
    def _iter_progress(iterable, desc, unit="file"):
        return iterable

# ------------------------------ UTIL ------------------------------
N_RE = re.compile(r"N=(\d+)", re.IGNORECASE)

def extract_N(path: Path) -> str:
    m = N_RE.search(path.name)
    return m.group(1) if m else "unknown"

def gather(dirpath: Path, pattern="*.csv"):
    if not dirpath.exists():
        return []
    return sorted(dirpath.glob(pattern))

def index_by_N(files):
    idx = defaultdict(list)
    for f in files:
        idx[extract_N(f)].append(f)
    return idx

def report_counts(title, files):
    byN = index_by_N(files)
    print(title + ":")
    if not byN:
        print("  0 files")
        return
    total = 0
    for key in sorted(byN, key=lambda x: (x=="unknown", int(x) if x.isdigit() else 0)):
        c = len(byN[key]); total += c
        print(f"  N={key}: {c} file(s)")
    print(f"  Total: {total} file(s)")

def copy_files(files, dest_dir: Path, label: str):
    dest_dir.mkdir(parents=True, exist_ok=True)
    counts = Counter()
    it = _iter_progress(files, desc=f"Copy {label}")
    copied = 0
    for src in it:
        dst = dest_dir / src.name
        if dst.exists():
            continue
        shutil.copy2(src, dst)
        counts[extract_N(src)] += 1
        copied += 1
    print(f"{label}: copied {copied} new file(s).")
    return counts

def select_with_N(files, N_val: int):
    return [f for f in files if extract_N(f) == str(N_val)]

# ----------------- NEW: CLEANUP OF TRAILING (1) FILES -----------------
# Matches filenames that end with " (1)" or " (1).csv" exactly
TRAILING_ONE_RE = re.compile(r"\s\(\s*1\s*\)(?:\.csv)?$", re.IGNORECASE)

def list_trailing_one_files(dirpath: Path, Ns=None):
    Ns = set(str(n) for n in (Ns or []))
    out = []
    for f in gather(dirpath, "*"):
        # Only consider CSV-like files or files without extension if they match
        name = f.name
        if not TRAILING_ONE_RE.search(name):
            continue
        if Ns:
            if extract_N(f) not in Ns:
                continue
        out.append(f)
    return out

def delete_with_progress(files, label: str):
    if not files:
        print(f"{label}: nothing to delete.")
        return 0
    it = _iter_progress(files, desc=f"Delete {label}", unit="file")
    count = 0
    for f in it:
        try:
            f.unlink(missing_ok=True)
            count += 1
        except Exception as e:
            print(f"Could not delete {f}: {e}")
    print(f"{label}: deleted {count} file(s).")
    return count

def cleanup_trailing_ones(main_dir: Path, raw_dir: Path, Ns):
    print("Scanning for trailing '(1)' duplicates in 5k folders...")
    main_bad = list_trailing_one_files(main_dir, Ns)
    raw_bad  = list_trailing_one_files(raw_dir,  Ns)

    # Delete MAIN first, then RAW
    deleted_main = delete_with_progress(main_bad, "5k MAIN '(1)'")
    deleted_raw  = delete_with_progress(raw_bad,  "5k RAW  '(1)'")

    # Report remaining
    print(f"Cleanup summary: MAIN deleted={deleted_main}, RAW deleted={deleted_raw}")

# ---------------------- GENERATION HOOK (USING YOUR HELPERS) ----------------------
# Uses your batch_transition_analysis(...) to create exactly ONE scenario per call.
# It picks a single M from MS on each call and varies the seed to avoid duplicates.

import random
from itertools import count

# --- CONFIG you can tweak to match your S1 generator defaults ---
MS = [7]                   # choose the M values you allow; one will be sampled per scenario
REQUIRE_ZOPA = True        # True to enforce a consensus node (uses your set_agreement_threshold)
RULE_CHOICE = 3            # 1 majority, 2 max-votes-minus-one, 3 unanimity (your default)
SAMPLING_PROPORTION = 1.0  # keep as in your current runs
BASE_SEED = 42             # per-call seeds = BASE_SEED + next(_seed_counter)

# Resolve your helper: prefer imported module, else fall back to global in this notebook
try:
    from transition_analysis import batch_transition_analysis as _bta  # if helpers are in a module
except Exception:
    if "batch_transition_analysis" in globals():
        _bta = globals()["batch_transition_analysis"]
    else:
        _bta = None

if _bta is None:
    raise RuntimeError(
        "Could not locate batch_transition_analysis. Make sure your helper "
        "functions are defined above or importable as transition_analysis."
    )

_seed_counter = count(0)

def generate_one_scenario(*, N: int, out_main_dir: Path, out_raw_dir: Path) -> None:
    """
    Create exactly one scenario CSV in out_main_dir (and raw in out_raw_dir) for this N.
    The filename pattern remains UM_N=<N>_M=<M>_<attempt>.csv as in your helper.
    """
    # pick ONE M per call so we generate exactly one scenario
    M = random.choice(MS)

    # vary seed so each invocation explores a different payoff space
    seed = BASE_SEED + next(_seed_counter)

    # Call your helper for a single scenario at (N, M)
    _bta(
        Ns=[N],
        Ms=[M],
        target_count=1,           # exactly one scenario
        raw_dir=str(out_raw_dir),
        out_dir=str(out_main_dir),
        rule_choice=RULE_CHOICE,
        require_zopa=REQUIRE_ZOPA,
        sampling_proportion=SAMPLING_PROPORTION,
        seed=seed,
    )

def _must_have_generator():
    # now satisfied by the wrapper above
    return

def generate_batch(N: int, k: int, out_main_dir: Path, out_raw_dir: Path):
    made = 0
    for _ in _iter_progress(range(k), desc=f"Generate N={N}", unit="scen"):
        generate_one_scenario(N=N, out_main_dir=out_main_dir, out_raw_dir=out_raw_dir)
        made += 1
    print(f"Generated {made} scenario(s) for N={N}.")


# ------------------------------ MODES ------------------------------
def mode_copy_from_old_main():
    print("Source (MAIN):", SRC_MAIN)
    print("Source (RAW): ", SRC_RAW if SRC_RAW.exists() else "(none)")
    print("Dest (MAIN):  ", S1_MAIN)
    print("Dest (RAW):   ", S1_RAW)

    main_files = gather(SRC_MAIN, "*.csv")
    raw_files  = gather(SRC_RAW,  "*.csv")

    copy_files(main_files, S1_MAIN, "MAIN CSVs")
    copy_files(raw_files,  S1_RAW,  "RAW CSVs")

    print("\n=== SUMMARY ===")
    report_counts("MAIN by N", gather(S1_MAIN))
    report_counts("RAW by N",  gather(S1_RAW))

def mode_build_5k_scenarios():
    print("Building 5k_scenarios bundle")
    print("Source S1 (MAIN):", S1_MAIN)
    print("Source S1 (RAW): ", S1_RAW)
    print("5k (MAIN):       ", FIVEK_MAIN)
    print("5k (RAW):        ", FIVEK_RAW)
    FIVEK_MAIN.mkdir(parents=True, exist_ok=True)
    FIVEK_RAW.mkdir(parents=True, exist_ok=True)

    # Step 0. Cleanup duplicates like "... (1)" or "... (1).csv" for selected Ns
    cleanup_trailing_ones(FIVEK_MAIN, FIVEK_RAW, NS)

    # Current state in 5k folder
    fivek_main_now = gather(FIVEK_MAIN)
    fivek_byN_now = index_by_N(fivek_main_now)

    # Step 1. Seed from S1_OUTPUT until at least MIN_BASE per N
    s1_main_all = gather(S1_MAIN)
    s1_raw_all  = gather(S1_RAW)

    for N in NS:
        have = len(fivek_byN_now.get(str(N), []))
        if have >= MIN_BASE:
            print(f"N={N}: already have {have} in 5k MAIN, meets base {MIN_BASE}.")
            continue

        s1_main_sel = select_with_N(s1_main_all, N)
        s1_raw_sel  = select_with_N(s1_raw_all,  N)

        existing_names_main = {p.name for p in gather(FIVEK_MAIN)}
        existing_names_raw  = {p.name for p in gather(FIVEK_RAW)}

        to_copy_main = [p for p in s1_main_sel if p.name not in existing_names_main]
        to_copy_raw  = [p for p in s1_raw_sel  if p.name not in existing_names_raw]

        print(f"N={N}: seeding from S1. Candidates MAIN={len(to_copy_main)}, RAW={len(to_copy_raw)}")
        copy_files(to_copy_main, FIVEK_MAIN, f"S1->5k MAIN N={N}")
        copy_files(to_copy_raw,  FIVEK_RAW,  f"S1->5k RAW  N={N}")

        fivek_main_now = gather(FIVEK_MAIN)
        fivek_byN_now = index_by_N(fivek_main_now)
        have = len(fivek_byN_now.get(str(N), []))
        print(f"N={N}: 5k MAIN after seed = {have} file(s).")

        if have < MIN_BASE:
            need = MIN_BASE - have
            print(f"N={N}: generating {need} to reach base {MIN_BASE}.")
            generate_batch(N, need, FIVEK_MAIN, FIVEK_RAW)

    # Step 2. Top up to TARGET_PER_N by generation
    fivek_main_now = gather(FIVEK_MAIN)
    fivek_byN_now = index_by_N(fivek_main_now)
    for N in NS:
        have = len(fivek_byN_now.get(str(N), []))
        if have >= TARGET_PER_N:
            print(f"N={N}: already at {have} >= target {TARGET_PER_N}.")
            continue
        need = TARGET_PER_N - have
        print(f"N={N}: generating {need} to reach target {TARGET_PER_N}.")
        generate_batch(N, need, FIVEK_MAIN, FIVEK_RAW)

    # Step 3. Final verification
    print("\n=== FINAL CHECK: 5k_scenarios MAIN by N ===")
    fivek_main_now = gather(FIVEK_MAIN)
    report_counts("5k MAIN by N", fivek_main_now)

    ok = True
    byN = index_by_N(fivek_main_now)
    for N in NS:
        c = len(byN.get(str(N), []))
        if c < TARGET_PER_N:
            print(f"[FAIL] N={N}: have {c}, need {TARGET_PER_N}.")
            ok = False
        else:
            print(f"[OK]   N={N}: have {c} files.")
    if not ok:
        raise SystemExit("5k build did not reach target for all N.")
    print("All targets met.")

# ------------------------------ RUN ------------------------------
if MODE == "copy_from_old_main":
    mode_copy_from_old_main()
elif MODE == "build_5k_scenarios":
    mode_build_5k_scenarios()
else:
    raise SystemExit(f"Unknown MODE={MODE}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Building 5k_scenarios bundle
Source S1 (MAIN): /content/drive/MyDrive/Negotiations-as-Search/S1_OUTPUT
Source S1 (RAW):  /content/drive/MyDrive/Negotiations-as-Search/S1_OUTPUT/random_raw
5k (MAIN):        /content/drive/MyDrive/Negotiations-as-Search/5k_scenarios
5k (RAW):         /content/drive/MyDrive/Negotiations-as-Search/5k_scenarios/random_raw
Scanning for trailing '(1)' duplicates in 5k folders...
5k MAIN '(1)': nothing to delete.
5k RAW  '(1)': nothing to delete.
Cleanup summary: MAIN deleted=0, RAW deleted=0
N=2: already have 2000 in 5k MAIN, meets base 2000.
N=2: generating 3000 to reach target 5000.


Streaming output truncated to the last 5000 lines.
Generate N=2:  44%|████▍     | 1334/3000 [33:03<38:18,  1.38s/scen]

Generate N=2:  44%|████▍     | 1335/3000 [33:04<38:23,  1.38s/scen]

Generate N=2:  45%|████▍     | 1336/3000 [33:06<38:54,  1.40s/scen]

Generate N=2:  45%|████▍     | 1337/3000 [33:07<38:38,  1.39s/scen]

Generate N=2:  45%|████▍     | 1338/3000 [33:08<38:20,  1.38s/scen]

Generate N=2:  45%|████▍     | 1339/3000 [33:10<38:31,  1.39s/scen]

Generate N=2:  45%|████▍     | 1340/3000 [33:11<38:35,  1.39s/scen]

Generate N=2:  45%|████▍     | 1341/3000 [33:13<38:31,  1.39s/scen]

Generate N=2:  45%|████▍     | 1342/3000 [33:14<38:46,  1.40s/scen]

Generate N=2:  45%|████▍     | 1343/3000 [33:15<38:33,  1.40s/scen]

Generate N=2:  45%|████▍     | 1344/3000 [33:17<38:48,  1.41s/scen]

Generate N=2:  45%|████▍     | 1345/3000 [33:18<38:40,  1.40s/scen]

Generate N=2:  45%|████▍     | 1346/3000 [33:20<38:45,  1.41s/scen]

Generate N=2:  45%|████▍     | 1347/3000 [33:21<38:2

Generated 3000 scenario(s) for N=2.

=== FINAL CHECK: 5k_scenarios MAIN by N ===
5k MAIN by N:
  N=2: 2000 file(s)
  N=3: 120 file(s)
  N=6: 2000 file(s)
  Total: 4120 file(s)
[FAIL] N=2: have 2000, need 5000.


SystemExit: 5k build did not reach target for all N.

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
